In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
company_page  = ['www.zalando.de','outfittery.de', 'www.amazon.de']
root_url = 'https://www.trustpilot.com/review/'
additive = '?page='

In [3]:
def form_other_pages_url(home_url,additive,count_range,company_url_list=[]):
  for count in count_range:
    if count == 1:
        other_page_link = home_url
    else:
        other_page_link = home_url + additive + str(count)
    company_url_list.append(other_page_link)
  return company_url_list
    
  

In [4]:
def get_url(count_range,company_url):
  home_url = root_url + company_url
  company_list = form_other_pages_url(home_url,additive,count_range,company_url_list=[])
  return company_list



In [5]:
zalando_urls = get_url(range(1,6),company_page[0])
amazon_urls = get_url(range(1,26),company_page[2])
outfittery_urls = get_url(range(1,6),company_page[1])

In [6]:
def get_reviews_objects(url):
    page_url= requests.get(url).text
    soup = BeautifulSoup(page_url, 'lxml')
    review_title = soup.find_all('a',class_='link link--large link--dark')
    review_body = soup.find_all('p',class_='review-content__text')
    return review_title,review_body

In [7]:
def url_list (url_list):
  apprend_content = []
  title_bulk = []
  body_bulk = []
  for url in url_list:
     review_title,review_body = get_reviews_objects(url)
     title_text = [review.text for review in review_title]
     body_text = [review.text for review in review_body]
     title_bulk += title_text
     body_bulk += body_text

  apprend_content.append([title_bulk,body_bulk])
  return apprend_content
  

In [8]:
zalando_content = url_list(zalando_urls)
amazon_content = url_list(amazon_urls)
outfittery_content = url_list(outfittery_urls)

In [9]:
data_base = pd.DataFrame()


In [12]:
map_company = {0:'zalando',1:'Amazon',2:'Outfittery'}
content_list = [zalando_content,amazon_content,outfittery_content]

for index,content in enumerate(content_list):
  data = pd.DataFrame(zip(content[0][0],content[0][1]))
  data['company'] = index
  data_base = pd.concat([data_base,data],ignore_index=True)


In [15]:
data_base['company'] = data_base['company'].map(map_company)
data_base.head()

In [17]:
data_base.to_csv('trustpilot_reviews.csv')